# Laser Cutting Tests

## Test Setup

# Code:

In [19]:
import os
import sys
from uuid import uuid4

import numpy as np

import GCode
import GRBL
from utils import picture

sys.path.append("..")

In [20]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.laser_mode = 1

print("Laser Mode: {}".format(cnc.laser_mode))


def init(feed=100):
    program = GCode.GCode()
    program.G21()  # Metric Units
    program.G90()  # Relative positioning.
    program.G92(X=0, Y=0, Z=0)  # Zero on where we put the workpiece
    program.G1(F=feed)
    return program


def end():
    program = GCode.GCode()
    program.M5()
    return program


ok
Laser Mode: 1.0


In [16]:
def laser_on(pwm=1):
    if int(pwm) != np.uint8(pwm):
        raise (Exception("UINT8! {}".format(pwm)))
    # Set minimal power setting to focus and position laser
    cnc.cmd("M3 S{:03d}".format(np.uint8(pwm)))
    cnc.cmd("G1 F10")  # Laser On


def laser_off():
    cnc.cmd("M5")  # Laser off


In [17]:
def pulse(pulse_duration=100):
    prog = GCode.GCode()
    prog.M5()
    prog.G91()
    prog.G0(X=0)
    prog.M3(S=255)
    prog.G1(X=0)
    prog.G4(U=pulse_duration)
    prog.G0(X=0)
    prog.G90()
    prog.M5()
    return prog


In [21]:
cnc.run(init() + pulse(100) + end())


1.4474668502807617

In [6]:
cnc.run(init() + pulse(500) + end())


1.2418253421783447

In [7]:
cnc.run(init() + pulse(1000) + end())


1.2426557540893555

In [22]:
test_run = GCode.GCode()
# TODO: Get z-axis probe.
test_run += init(feed=200)
for i in range(3, 10):
    Xs = np.linspace(0, 10, i)
    for X in Xs:
        test_run.G0(X=np.round(X, 4))
        test_run += pulse(1)


In [23]:
test_run


<GCode>[cmds=424]

In [24]:
gcode_file = "LaserCut.gcode"


In [25]:
test_run.save(gcode_file)

del test_run
test_run = GCode.GCode()

test_run.load(gcode_file)


In [26]:
test_run.buffer[0:5]


['G21', 'G90', 'G92 X0 Y0 Z0', 'G1 F200', 'G0 X0.0']

In [27]:
from time import sleep

In [28]:
while 1:
    try:
        cnc.run(test_run)
        while 1:
            print(cnc.status)
            sleep(5)
    except KeyboardInterrupt as error:
        print("Feed Hold")
        cnc.cmd("!")
        while 1:
            try:
                cnc.reset()
                break
            except:
                pass
        print("^C")
        break
    except:
        raise


<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0|Ov:100,100,100>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0>
<Idle|MPos:10.000,0.000,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>
Feed Hold
^C
